In [ ]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from utils import GLOBAL, functions

In [ ]:
import pandas as pd

In [ ]:
proj_df = pd.read_csv(GLOBAL.PROJECTIONS_2020, index_col=0)

In [ ]:
### DO NOT RUN IF USING 2021 PROJECTIONS OR HIGHER ####
### Section is for older projections newer ones have the scoring columns ###

#Get rid of odd column if there are any[row indexer, column indexer], change 0 to 1 if there are
proj_df = proj_df.iloc[:, 0:]

#Add in Fantasy Points based on scoring type
proj_df['FantasyPoints'] = (
    functions.scoringCalculator(
        proj_df['Receptions'], proj_df['ReceivingYds'], proj_df['ReceivingTD'], proj_df['FL'], proj_df['RushingYds'], proj_df['RushingTD'], proj_df['PassingYds'], proj_df['PassingTD'], proj_df['Int'], GLOBAL.HALF_PPR_SCORING
    )
)

In [ ]:
columns = ['Player', 'Team', 'Pos', 'FantasyPoints', 'Receptions', 'ReceivingYds', 'ReceivingTD', 'RushingAtt', 'RushingYds', 'RushingTD', 'FL']

#Get dataframes of specific pos
rb_proj_df = proj_df.loc[(proj_df['Pos'] == 'RB', columns)]
wr_proj_df = proj_df.loc[(proj_df['Pos'] == 'WR', columns)]
te_proj_df = proj_df.loc[(proj_df['Pos'] == 'TE', columns)]
qb_proj_df = proj_df.loc[(proj_df['Pos'] == 'QB', columns)]

In [ ]:
#ADP for current year and scoring format
adp_df = pd.read_csv(GLOBAL.ADP_2020_HALF_PPR, index_col=0)

adp_df['ADP RANK'] = adp_df['AVG'].rank()

adp_df_cutoff = adp_df[:100]


replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
}

for _, row in adp_df_cutoff.iterrows():
    
    position = row['POS'] # extract out the position and player value from each row as we loop through it
    player = row['PLAYER']
    
    if position in replacement_players: # if the position is in the dict's keys
        replacement_players[position] = player # set that player as the replacement player

vor_df = proj_df[['Player', 'Pos', 'Team', 'FantasyPoints']]

In [ ]:
replacement_values = {} # initialize an empty dictionary

for position, player_name in replacement_players.items():
    
    player = proj_df.loc[proj_df['Player'] == player_name]
    
    # because this is a series object we get back, we need to use the tolist method
    # to get back the series as a list. The list object is of length 1, and the 1 item has the value we need.
    # we tack on a [0] to get the value we need.
    
    replacement_values[position] = player['FantasyPoints'].tolist()[0]

In [ ]:
pd.set_option('chained_assignment', None)

vor_df = vor_df.loc[vor_df['Pos'].isin(['QB', 'RB', 'WR', 'TE'])]

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
vor_df['VOR'] = vor_df.apply(
    lambda row: row['FantasyPoints'] - replacement_values.get(row['Pos']), axis=1
)

In [ ]:
pd.set_option('display.max_rows', None) # turn off truncation of rows setting inherent to pandas

vor_df['VOR Rank'] = vor_df['VOR'].rank(ascending=False)
vor_df.sort_values(by='VOR', ascending=False).head(100)

In [ ]:
vor_df.groupby('Pos')['VOR'].describe()

In [ ]:
vor_df['VOR'] = vor_df['VOR'].apply(lambda x: (x - vor_df['VOR'].min()) / (vor_df['VOR'].max() - vor_df['VOR'].min()))

In [ ]:
vor_df = vor_df.sort_values(by='VOR Rank')
vor_df.head(100)

In [ ]:
import seaborn as sns # in case you did not import it above

# calculating how many players are in our draft pool.
num_teams = 12
num_spots = 16 # 1 QB, 2RB, 2WR, 1TE, 1FLEX, 1K, 1DST, 7 BENCH
draft_pool = num_teams * num_spots

df_copy =vor_df[:draft_pool]

sns.boxplot(x=df_copy['Pos'], y=df_copy['VOR']);

In [ ]:
vor_df = vor_df.rename({
    'VOR': 'Value',
    'VOR Rank': 'Value Rank'
}, axis=1) # axis = 1 means make the change along the column axis.

In [ ]:
adp_df = adp_df.rename({
    'PLAYER': 'Player',
    'POS': 'Pos',
    'AVG': 'Average ADP',
    'ADP RANK': 'ADP Rank'
}, axis=1) # let's rename some columns first.

In [ ]:
final_df = vor_df.merge(adp_df, how='left', on=['Player', 'Pos'])

final_df.head()

In [ ]:
final_df['Diff in ADP and Value'] = final_df['ADP Rank'] - final_df['Value Rank']
final_df.head()

In [ ]:
draft_pool = final_df.sort_values(by='ADP Rank')[:196]

rb_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'RB']
qb_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'QB']
wr_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'WR']
te_draft_pool = draft_pool.loc[draft_pool['Pos'] == 'TE']

In [ ]:
# top 10 RB sleepers for this year's draft
rb_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False)[:10]

In [ ]:
# top 10 RB overvalued for this year's draft
rb_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True)[:10]

In [ ]:
# top 10 WR sleepers for this year's draft
wr_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False)[:10]

In [90]:
# top 10 WR overvalued for this year's draft
wr_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True)[:10]

,Player,Pos,Team,FantasyPoints,Value,Value Rank,Average ADP,ADP Rank,Diff in ADP and Value
186,Alshon Jeffery,WR,PHI,92.07,0.469033,187.0,122.0,118.0,-69.0
244,Denzel Mims,WR,NYJ,62.66,0.403989,245.0,175.0,186.5,-58.5
272,Devin Duvernay,WR,BAL,52.55,0.381629,273.0,216.0,225.0,-48.0
138,Jerry Jeudy,WR,DEN,121.58,0.534298,139.0,116.5,109.0,-30.0
108,Deebo Samuel,WR,SF,136.55,0.567406,109.0,82.0,80.0,-29.0
167,Mecole Hardman,WR,KC,105.23,0.498138,168.0,146.5,143.0,-25.0
45,Odell Beckham Jr.,WR,CLE,184.07,0.672502,46.0,25.5,27.0,-19.0
46,JuJu Smith-Schuster,WR,PIT,182.94,0.670003,47.0,32.0,30.0,-17.0
66,D.J. Chark,WR,JAC,167.06,0.634883,67.0,52.0,50.5,-16.5
123,Emmanuel Sanders,WR,NO,127.89,0.548253,124.0,117.0,111.0,-13.0


In [91]:
# top 10 TE sleepers for this year's draft
te_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False)[:10]

,Player,Pos,Team,FantasyPoints,Value,Value Rank,Average ADP,ADP Rank,Diff in ADP and Value
116,Chris Herndon IV,TE,NYJ,108.81,0.556812,117.0,178.0,189.0,72.0
131,Blake Jarwin,TE,DAL,100.70,0.538876,132.0,166.5,172.0,40.0
120,Dallas Goedert,TE,PHI,105.96,0.550509,121.0,158.5,160.0,39.0
177,Jimmy Graham,TE,CHI,75.85,0.483917,178.0,202.5,211.0,33.0
137,Eric Ebron,TE,PIT,98.80,0.534674,138.0,163.0,169.0,31.0
127,Jonnu Smith,TE,TEN,103.74,0.545599,128.0,157.0,157.5,29.5
173,Irv Smith Jr.,TE,MIN,76.39,0.485111,174.0,192.0,201.0,27.0
135,Ian Thomas,TE,CAR,99.52,0.536266,136.0,159.0,161.5,25.5
157,Greg Olsen,TE,SEA,89.04,0.513088,158.0,170.5,180.0,22.0
103,Mike Gesicki,TE,MIA,116.07,0.572869,104.0,130.0,125.5,21.5


In [92]:
# top 10 TE overvalued for this year's draft
te_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True)[:10]

,Player,Pos,Team,FantasyPoints,Value,Value Rank,Average ADP,ADP Rank,Diff in ADP and Value
104,Rob Gronkowski,TE,TB,115.95,0.572603,105.0,83.0,81.0,-24.0
82,Tyler Higbee,TE,LAR,129.06,0.601598,83.0,70.0,67.0,-16.0
80,Hunter Henry,TE,LAC,131.44,0.606861,81.0,68.5,66.0,-15.0
47,Mark Andrews,TE,BAL,156.78,0.662904,48.0,36.5,35.5,-12.5
90,Jared Cook,TE,NO,122.15,0.586315,91.0,80.5,78.5,-12.5
100,Austin Hooper,TE,CLE,116.37,0.573532,101.5,103.0,98.0,-3.5
21,George Kittle,TE,SF,191.19,0.739006,22.0,19.5,20.0,-2.0
40,Zach Ertz,TE,PHI,168.57,0.688979,41.0,41.5,41.0,0.0
53,Darren Waller,TE,LV,152.32,0.653040,54.0,54.0,55.5,1.5
12,Travis Kelce,TE,KC,208.24,0.776714,13.0,17.0,15.5,2.5


In [93]:
# top 10 QB sleepers for this year's draft
qb_draft_pool.sort_values(by='Diff in ADP and Value', ascending=False)[:10]

,Player,Pos,Team,FantasyPoints,Value,Value Rank,Average ADP,ADP Rank,Diff in ADP and Value
86,Cam Newton,QB,NE,267.554,0.591729,87.0,203.5,213.0,126.0
97,Jared Goff,QB,LAR,260.608,0.576367,98.0,153.5,153.0,55.0
105,Ryan Tannehill,QB,TEN,258.048,0.570706,106.0,147.0,144.0,38.0
129,Kirk Cousins,QB,MIN,245.184,0.542255,130.0,162.0,167.5,37.5
112,Daniel Jones,QB,NYG,253.094,0.559749,113.0,144.0,139.0,26.0
36,Deshaun Watson,QB,HOU,312.888,0.691991,37.0,57.5,59.0,22.0
118,Gardner Minshew II,QB,JAC,251.500,0.556224,119.0,145.0,141.0,22.0
143,Teddy Bridgewater,QB,CAR,239.328,0.529304,144.0,160.5,165.5,21.5
50,Josh Allen,QB,BUF,297.482,0.657919,51.0,70.5,68.0,17.0
30,Dak Prescott,QB,DAL,318.416,0.704217,31.0,49.0,47.0,16.0


In [94]:
# top 10 QB overvalued for this year's draft
qb_draft_pool.sort_values(by='Diff in ADP and Value', ascending=True)[:10]

,Player,Pos,Team,FantasyPoints,Value,Value Rank,Average ADP,ADP Rank,Diff in ADP and Value
126,Ben Roethlisberger,QB,PIT,246.772,0.545767,127.0,120.5,115.5,-11.5
102,Matthew Stafford,QB,DET,259.326,0.573532,101.5,104.0,99.0,-2.5
155,Philip Rivers,QB,IND,232.434,0.514057,156.0,154.0,154.0,-2.0
17,Patrick Mahomes,QB,KC,345.384,0.763860,18.0,18.0,17.0,-1.0
70,Drew Brees,QB,NO,279.616,0.618406,71.0,75.0,72.5,1.5
130,Joe Burrow,QB,CIN,243.696,0.538964,131.0,138.5,134.5,3.5
52,Kyler Murray,QB,ARI,295.652,0.653872,53.0,54.5,57.0,4.0
76,Tom Brady,QB,TB,276.228,0.610913,77.0,86.5,83.0,6.0
117,Baker Mayfield,QB,CLE,251.740,0.556755,118.0,130.0,125.5,7.5
79,Carson Wentz,QB,PHI,274.890,0.607954,80.0,95.0,88.5,8.5
